In [ ]:
#Imports
import numpy as np
import datetime as dt

from eodag import EODataAccessGateway
from eodag import setup_logging

from rasterio.crs import CRS

import rioxarray
import rasterio as rio


import eotools.loading as eoload
import eotools.shortcut as eoshort 
import eotools.regions as eoregion

# Setup Verbose Values:
# 0: no logging and no progress bar
# 1: no logging but progress bars displayed
# 2: log at the INFO level
# 3: log at the DEBUG level (even more information)

setup_logging(verbose=1)

# EODAG - Regions of Interest
This Notebook is intended to demonstrate how to make `geojson` Files containing polygons with Regions of Interest.

In [ ]:
# Get Credentials from .env file and make dirs from paths.yml
# These functions are just shortcuts from a python script and just help to keep the notebooks short and simple
workspace = eoshort.read_paths(filepath='paths.yml')
dag = EODataAccessGateway()
dag.set_preferred_provider("cop_dataspace")
deserialized_search_results = eoshort.deserialize(filepath="search_results.geojson", workspace=workspace, dag=dag, log=True)

# Download Single Product
product = deserialized_search_results[1]
path = dag.download(product)

In [ ]:
# Set Boundingbox for Area inside the Tile.
latmin, latmax = 48.1, 48.35
lonmin, lonmax = 16.1, 16.6
extent = {'lonmin': lonmin, 'latmin': latmin, 'lonmax': lonmax, 'latmax': latmax}

# Setting common Parameters for all further image processing
common_params = dict(
    crs=CRS.from_epsg(4326),               # the downloaded images are in 4326, don't reproject them
    resolution=0.0001,                     # but lower their resolution (0.0006 should be 60m in 100km)
    extent=(lonmin,latmin,lonmax,latmax)   # and zoom over/crop the area of interest
)

In [ ]:
# There has been some error with eodag when loading some data into an xarray
# There might be some processing pipelines which result in eodag grabbing the wrong file.
# In such a case try to use the following command to load the dataset

# assets = eoload.get_assets(path, res=10)
# ds = eoload.load_single_product_regex(product=product, bands=assets, **common_params)


In [ ]:
# Get List of all available assets from a product
bands_2_load = eoload.get_assets(path, res=10, only_spectral=True, include_tci=True)

# Loading multiple Bands into a dataset
ds = eoload.load_single_product(product=product, bands=bands_2_load, params=common_params)

# Selecting a single Timestamp from the Dataset
single_img = ds.sel(time=dt.datetime(2021, 4, 12), method='nearest')

In [ ]:
# Activate the widget magic 
%matplotlib widget 
%matplotlib widget 

# Use roi to draw polygons and export them.
eoregion.roi(canvas=np.transpose(single_img['TCI'].values, (1,2,0)), ds=ds, button_1='Urban', button_2='NonUrban', figsize=(12,12), savepath='../data/shapefiles')